In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [10]:
pd.options.display.max_columns = None
data_explain = pd.read_excel("Winter 2023 Showcase Data Dictionary.xlsx")
data_explain

,Problem Statement,Unnamed: 1,Unnamed: 2
0,Many customers decide to trade-in their curren...,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,Data set description,NaN,NaN
5,The data is composed of vehicle attributes vis...,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,Column Name,Data Type,Description
9,price,String,"The price of the customer's purchased vehicle,..."


In [11]:
df = pd.read_csv("ShowcaseDataWinter2023/ShowcaseDataWinter2023.csv")
df

,price,appraisal_offer,online_appraisal_flag,model_year,mileage,make,model,trim_descrip,body,color,engine,cylinders,mpg_city,mpg_highway,horsepower,fuel_capacity,model_year_appraisal,mileage_appraisal,make_appraisal,model_appraisal,trim_descrip_appraisal,body_appraisal,color_appraisal,engine_appraisal,cylinders_appraisal,mpg_city_appraisal,mpg_highway_appraisal,horsepower_appraisal,fuel_capacity_appraisal,market
0,$20k to $25k,$0k to $5k,1.0,2014.0,15k to 20k miles,HDT,HDT_12,Premium,E,Red,2.0L,4,27.0,33.0,152,16.0,2001.0,200k+ miles,AIH,AIH_32,NaN,D,Silver,4.0L,6,17.0,21,236,21.0,13
1,$15k to $20k,$0k to $5k,0.0,2006.0,100k+ miles,KQZ,KQZ_23,Premium,E,Silver,3.5L,6,19.0,27.0,285,18.0,1996.0,0 to 10k miles,KQZ,KQZ_3,Not Premium,D,Gold,4.2L,6,16.0,21,205,25.0,11
2,$30k to $35k,$35k to $40k,1.0,2014.0,5k to 10k miles,CZV,CZV_10,Premium,E,Black,2.5L,4,24.0,30.0,187,15.0,2014.0,10k to 20k miles,CIP,CIP_16,NaN,E,Red,2.0L,4,41.0,38,181,10.0,13
3,$35k to $40k,$5k to $10k,1.0,2014.0,15k to 20k miles,IZA,IZA_19,Premium,F,Silver,2.0L,4,25.0,34.0,255,15.0,2006.0,100k to 110k miles,IZA,NaN,Premium,F,Blue,3.0L,6,17.0,25,230,16.0,13
4,$30k to $35k,$15k to $20k,1.0,2011.0,35k to 40k miles,HDT,HDT_8,Premium,F,Blue,2.5L,4,17.0,23.0,305,15.0,2011.0,20k to 30k miles,LTX,LTX_20,Not Premium,E,Black,2.4L,4,20.0,29,182,18.0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,$15k to $20k,$15k to $20k,1.0,2013.0,10k to 15k miles,OFW,OFW_6,Premium,F,Black,2.0L,4,28.0,37.0,147,14.0,2015.0,0 to 10k miles,LTX,LTX_30,Not Premium,E,Black,1.4L,4,26.0,31,138,14.0,1
199996,$20k to $25k,$10k to $15k,0.0,2014.0,55k to 60k miles,ARU,ARU_12,Not Premium,F,White,2.5L,4,27.0,37.0,182,16.0,2010.0,50k to 60k miles,HXQ,HXQ_8,Not Premium,B,Black,1.8L,4,30.0,39,143,13.0,8
199997,$30k to $35k,$0k to $5k,0.0,2014.0,10k to 15k miles,AIH,AIH_13,Premium,F,Silver,2.5L,4,28.0,39.0,206,16.0,2004.0,120k to 130k miles,HXQ,HXQ_14,Not Premium,F,Gray,2.4L,4,21.0,30,177,18.0,3
199998,$30k to $35k,$10k to $15k,0.0,2012.0,50k to 55k miles,KQZ,KQZ_3,Not Premium,D,Black,3.5L,6,17.0,23.0,282,23.0,2006.0,130k to 140k miles,BZQ,BZQ_4,Premium,D,White,5.7L,8,13.0,19,390,26.0,11


# Things to do:

- Price and appraisal_offer seem to be the 'target' type columns

- Depending on binary record 'online_apprasial_flag', should all feature columns be verified with the online and appraisal values?

- Convert relevant string data into numerical 
    - price
    - appraisal_offer
    - mileage
    - milage_appraisal
    
- Change the following data types
    - online_appraisal_flag -> to int
    - model_year -> to int
    - enging -> to int (given all values in X.XL format)
    
- Make/Model abbreviations are not trivial
- Check for NaNs on all columns
- 'Market' is a categorical variable?